### 0. Importing modules and data

In [99]:
import pandas as pd
from edastatmil_milser import edas_tatmil as EDA
import os
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
movies_df = pd.read_csv('../data/raw/Movies.csv')
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [82]:
credits_df = pd.read_csv('../data/raw/Credits.csv')
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Merge the dataframes

In [83]:
raw_df = pd.merge(movies_df, credits_df, how='inner', left_on='id', right_on='movie_id')
raw_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [84]:
print(movies_df.shape)
print(credits_df.shape)
print(raw_df.shape)

(4803, 20)
(4803, 4)
(4803, 24)


### 1. Transform data
#### 1.1 Drop innecesary columns

In [85]:
df = raw_df.copy()
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')

In [86]:
columns = ['movie_id','original_title','overview','genres','keywords','cast','crew']
df = df[columns]
df.columns

Index(['movie_id', 'original_title', 'overview', 'genres', 'keywords', 'cast',
       'crew'],
      dtype='object')

#### 1.2 Get info from json

In [87]:
def getname(json_str):
    json_data = json.loads(json_str)
    names = [item['name'] for item in json_data]
    return ','.join(names)

def get3(text):
    text_list = text.split(',')
    to_column = text_list[:3]
    return ','.join(to_column)

def getdirector(crew_json):
    crew_data = json.loads(crew_json)
    director = next((crew for crew in crew_data if crew.get('job') == 'Director'), None)
    return director.get('name') if director else None #if there is no director, None


In [88]:
df['genres'] = df['genres'].apply(getname)
df['keywords'] = df['keywords'].apply(getname)
df['cast'] = df['cast'].apply(getname)

df['cast'] = df['cast'].apply(get3)

df['director'] = df['crew'].apply(getdirector)
df = df.drop(columns=['crew'])

df.head()

,movie_id,original_title,overview,genres,keywords,cast,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action,Adventure,Fantasy,Science Fiction","culture clash,future,space war,space colony,so...","Sam Worthington,Zoe Saldana,Sigourney Weaver",James Cameron
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Adventure,Fantasy,Action","ocean,drug abuse,exotic island,east india trad...","Johnny Depp,Orlando Bloom,Keira Knightley",Gore Verbinski
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"Action,Adventure,Crime","spy,based on novel,secret agent,sequel,mi6,bri...","Daniel Craig,Christoph Waltz,Léa Seydoux",Sam Mendes
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"Action,Crime,Drama,Thriller","dc comics,crime fighter,terrorist,secret ident...","Christian Bale,Michael Caine,Gary Oldman",Christopher Nolan
4,49529,John Carter,"John Carter is a war-weary, former military ca...","Action,Adventure,Science Fiction","based on novel,mars,medallion,space travel,pri...","Taylor Kitsch,Lynn Collins,Samantha Morton",Andrew Stanton


#### 1.2 Null values

In [89]:
c,n = EDA.explore(df)

Rows: 4803
Columns: 7
+----------------+----------------+------------+-----------+---------------+
|                | Non-Null Count | Null Count | Data Type | Data Category |
+----------------+----------------+------------+-----------+---------------+
|    movie_id    |      4803      |     0      |   int64   |   Numerical   |
| original_title |      4803      |     0      |  object   |  Categorical  |
|    overview    |      4800      |     3      |  object   |  Categorical  |
|     genres     |      4803      |     0      |  object   |  Categorical  |
|    keywords    |      4803      |     0      |  object   |  Categorical  |
|      cast      |      4803      |     0      |  object   |  Categorical  |
|    director    |      4773      |     30     |  object   |  Categorical  |
+----------------+----------------+------------+-----------+---------------+


No director found for 30 elements, which we will remove from the dataset. We will do the same with the elements without overview.

In [90]:
df = df.dropna(subset=['director', 'overview'])

#### 1.4 Drop spaces from names

In [91]:
df['cast'] = df['cast'].str.replace(' ', '')
df['director'] = df['director'].str.replace(' ', '')
df['keywords'] = df['keywords'].str.replace(' ', '')
print(df.loc[0, 'cast'])

SamWorthington,ZoeSaldana,SigourneyWeaver


#### 1.5 Make list of words in each column

In [92]:
def split_list(column, delimiter, df):
    df_ = df.copy()
    df_[column] = df_[column].apply(lambda x: x.split(delimiter) if isinstance(x, str) else np.nan)
    return df_

In [93]:
df = split_list('overview',' ',df)

columns2 = ['keywords','genres','cast']
for c in columns2:
    df = split_list(c, ',', df)

In [94]:
print(df.loc[0, 'overview'])
print(df.loc[0, 'genres'])
print(df.loc[0, 'keywords'])
print(df.loc[0, 'cast'])
print(df.loc[0, 'director'])

['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.']
['Action', 'Adventure', 'Fantasy', 'Science Fiction']
['cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d']
['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']
JamesCameron


#### 1.6 Column tag

In [95]:
combined_series = df['overview'] + df['genres'] + df['keywords'] + df['cast'] 

df['tags'] = combined_series.apply(lambda lst: ' '.join(lst))
df['tags'] = df['tags'] + ' ' + df['director']

print(df.loc[0, 'tags'])

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron


In [96]:
df.to_csv('../data/processed/Processed.csv')

### 2. KNN
#### 2.1 Vectorize

In [97]:
vec_df = df['tags']

In [100]:
vec_model = CountVectorizer(stop_words = "english")
vec_df = vec_model.fit_transform(vec_df).toarray()

#### 2.2 Distance

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vec_df)

#### 2.3 Find 5 similar films

In [102]:
def recommend(movie):
    movie_index = df[df["original_title"] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(df.iloc[i[0]].original_title)

In [105]:
recommend('Escape from Planet Earth')

Planet 51
Titan A.E.
Meet Dave
Beneath the Planet of the Apes
Riddick
